In [1]:
## 데이터 스케일링과 문자열 처리 파이프라인

##pdpipe에는 Scikit_learn의 스케일러 메소드와 NLTK의 문자열 처리 메소드가 내장되어 있다. 

In [2]:
import pandas as pd
import pdpipe as pdp

df = pd.read_csv('./USA_Housing.csv')

def size(n):
    if n <= 6.3:
        return 'Small'
    elif 6.3 < n <= 7.7:
        return 'Medium'
    else:
        return 'Big'

df['House_size'] = df['Avg. Area Number of Rooms'].apply(size)

pipeline = pdp.ColDrop('Avg. Area Number of Rooms')
pipeline += pdp.OneHotEncode('House_size')
pipeline += pdp.RowDrop({'Price': lambda x: x <= 250000})

df2 = pipeline(df)

print('Number of Rows : ', len(df2), sep='')
df2.head()

Number of Rows : 4990


,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Bedrooms,Area Population,Price,Address,House_size_Medium,House_size_Small
0,79545.458574,5.682861,4.09,23086.800503,1.059034e+06,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701...",1,0
1,79248.642455,6.002900,3.09,40173.072174,1.505891e+06,"188 Johnson Views Suite 079\nLake Kathleen, CA...",1,0
2,61287.067179,5.865890,5.13,36882.159400,1.058988e+06,"9127 Elizabeth Stravenue\nDanieltown, WI 06482...",0,0
3,63345.240046,7.188236,3.26,34310.242831,1.260617e+06,USS Barnett\nFPO AP 44820,0,1
4,59982.197226,5.040555,4.23,26354.109472,6.309435e+05,USNS Raymond\nFPO AE 09386,0,0


In [3]:
## 스케일링 기능 
## Standardization Scaling : 평균을 0, 분산을 1
## Min-Max Scaling : 0과 1사이에 위치

In [4]:
pipeline_scaler = pdp.Scale('StandardScaler', exclude_columns=['House_size_Medium', 'House_size_Small'])

df3 = pipeline_scaler(df2)

In [5]:
df3.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Bedrooms,Area Population,Price,Address,House_size_Medium,House_size_Small
0,1.028113,-0.300130,0.087245,-1.322810,-0.500532,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701...",0.980157,-0.576579
1,1.000175,0.023142,-0.722671,0.401923,0.775998,"188 Johnson Views Suite 079\nLake Kathleen, CA...",0.980157,-0.576579
2,-0.690443,-0.115252,0.929559,0.069730,-0.500662,"9127 Elizabeth Stravenue\nDanieltown, WI 06482...",-1.020245,-0.576579
3,-0.496719,1.220456,-0.584986,-0.189886,0.075327,USS Barnett\nFPO AP 44820,-1.020245,1.734368
4,-0.813263,-0.948927,0.200634,-0.992999,-1.723449,USNS Raymond\nFPO AE 09386,-1.020245,-0.576579


In [6]:
df3.describe()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Bedrooms,Area Population,Price,House_size_Medium,House_size_Small
count,4.990000e+03,4.990000e+03,4.990000e+03,4.990000e+03,4.990000e+03,4.990000e+03,4.990000e+03
mean,-1.242081e-15,1.231240e-16,-4.534894e-17,-2.937752e-16,-7.796035e-17,1.090199e-16,-2.067818e-16
std,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00
min,-4.783945e+00,-3.369390e+00,-1.605481e+00,-3.635828e+00,-2.802583e+00,-1.020245e+00,-5.765789e-01
25%,-6.699448e-01,-6.587607e-01,-6.821755e-01,-6.830938e-01,-6.712406e-01,-1.020245e+00,-5.765789e-01
50%,1.881183e-02,-5.282162e-03,5.484873e-02,2.692235e-03,-2.957818e-03,9.801568e-01,-5.765789e-01
75%,6.750385e-01,6.791061e-01,4.112121e-01,6.749146e-01,6.785985e-01,9.801568e-01,-5.765789e-01
max,3.678300e+00,3.574860e+00,2.039145e+00,3.374546e+00,3.527482e+00,9.801568e-01,1.734368e+00


In [7]:
## 문자열 처리 기능 추가
## Address를 ZIP코드나 주(state)를 추출

In [8]:
df.loc[0,'Address']

'208 Michael Ferry Apt. 674\nLaurabury, NE 37010-5101'

In [9]:
def extract_state(token) :
    return str(token[-2])

pipeline_tokenizer = pdp.TokenizeText('Address') ## 토큰화
pipeline_state = pdp.ApplyByCols('Address', extract_state, result_columns='State')
pipeline_state_extract = pipeline_tokenizer + pipeline_state

df4 = pipeline_state_extract(df3)

In [10]:
df4.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Bedrooms,Area Population,Price,State,House_size_Medium,House_size_Small
0,1.028113,-0.300130,0.087245,-1.322810,-0.500532,NE,0.980157,-0.576579
1,1.000175,0.023142,-0.722671,0.401923,0.775998,CA,0.980157,-0.576579
2,-0.690443,-0.115252,0.929559,0.069730,-0.500662,WI,-1.020245,-0.576579
3,-0.496719,1.220456,-0.584986,-0.189886,0.075327,AP,-1.020245,1.734368
4,-0.813263,-0.948927,0.200634,-0.992999,-1.723449,AE,-1.020245,-0.576579
